In [8]:

##### NOTE : Need to fix this import method
from design import *
from lab import *
from lims import *
#from src.data.utils import bioimager, imagingStation, nanophotometer, figures
#from IPython.core.display import Image

ModuleNotFoundError: No module named 'design'

In [11]:
import pydna
import Bio

In [12]:
from pydna.dseqrecord import Dseqrecord
from pydna.dseq import Dseq
from Bio.Restriction import AsiSI, BsmI
from pydna.parsers import parse_primers
from pydna.amplify import pcr
from pydna.amplicon import Amplicon
from pydna.assembly import Assembly
from pydna.primer import Primer
from pydna.gel import gel
# see ladders.py
from pydna.ladders import *


# Project

Debottlenecking the strictosidine module

Hypothesis
1. Specific combinations of CPR / G8H homologs and corresponding expression levels can remove the G8H bottleneck in the Strictosidine pathway

Aim: To test the hypothesis

Tasks
1. **dgRNA**
    - to knockout G8H and CPR in MIA-CM-3
2. Base strain
3. Library
4. Phenotyping
5. Data analysis

### What gRNAs


G8H/CPR
- Both leu marker
- Make  double gRNA p1_G09 (pESC-LEU-gRNA_ATF1_CroCPR) from
    - PL_05_D9 (pESC-LEU-gRNA_ATF1)
    - p1_F04 (pESC-LEU-gRNA_CroCPR)

In [13]:
from IPython import display
display.Image('../pictures/Construct multiple gRNA.png')

FileNotFoundError: No such file or directory: '../pictures/Construct multiple gRNA.png'

FileNotFoundError: No such file or directory: '../pictures/Construct multiple gRNA.png'

<IPython.core.display.Image object>

# Plasmid construction

## Design

In [14]:
# Most likly grna1 = pCfB-56(pESC-LEU-gRNA_ATF1-CroCPR)
#gRNA1_template_names    = ['pL_05_D9\(pRS425-gRNA_ATF1)']
gRNA1_template_names= ['pESC-LEU-gRNA_CroCPR-2']
gRNA2_template_names    = ['pESC-LEU-gRNA_CroCPR-2']
backbone_template_names = ['p0056\(pESC-LEU-ccdB-USER)']

In [15]:
##### NOTE : Need to fix this import method
import sys
sys.path.append('../constrain/lims')

from benchling_api import from_benchling

In [16]:
gRNA1_template    = from_benchling(gRNA1_template_names)
gRNA2_template    = from_benchling(gRNA2_template_names)
vector = Dseqrecord(from_benchling(backbone_template_names), circular = True)

In [17]:
gRNA2_template

SeqRecord(seq=Seq('TCGCGCGTTTCGGTGATGACGGTGAAAACCTCTGACACATGCAGCTCCCGGAGA...GTC'), id='seq_Qy1hV9Gf', name=['pESC-LEU-gRNA_CroCPR-2'], description='<unknown description>', dbxrefs=[])

In [18]:
vector.name = "p0056\(pESC-LEU-ccdB-USER)"
vector.annotations['batches'] = [{'location'     :'l4_I06',
                                       'volume'       : 50,
                                       'concentration': 200}]

In [19]:
from pydna.parsers import parse_primers
U_pSNR52_Fw_1,TJOS_66_P2R,U_pSNR52_Fw_2,TJOS_65_P1R = parse_primers(''' >U_pSNR52_Fw_1
                                      CGTGCGAUTCTTTGAAAAGATAATGTATGA

                                      >TJOS_66_P2R
                                      ACCTGCACUTAACTAATTACATGACTCGA 
                                      
                                      >U_pSNR52_Fw_2
                                      AGTGCAGGUTCTTTGAAAAGATAATGTATGA
                                      
                                      >TJOS_65_P1R
                                      CACGCGAUTAACTAATTACATGACTCGA''')

In [20]:
from pydna.amplify import pcr
gRNA1_pcr_prod = pcr(U_pSNR52_Fw_1,TJOS_66_P2R, gRNA1_template)
gRNA2_pcr_prod = pcr(U_pSNR52_Fw_2,TJOS_65_P1R, gRNA2_template)

In [21]:
print(len(gRNA1_pcr_prod))
print(len(gRNA2_pcr_prod))
print(gRNA1_pcr_prod.seq)
print(gRNA2_pcr_prod.seq)

425
425
CGTGCGATTCTTTGAAAAGATAATGTATGATTATGCTTTCACTCATATTTATACAGAAACTTGATGTTTTCTTTCGAGTATATACAAGGTGATTACATGTACGTTTGAAGTACAACTCTAGATTTTGTAGTGCCCTCTTGGGCTAGCGGTAAAGGTGCGCATTTTTTCACACCCTACAATGTTCTGTTCAAAAGATTTTGGTCAAACGCTGTAGAAGTGAAAGTTGGTGCGCATGTTTCGGCGTTCGAAACTTCTCCGCAGTGAAAGATAAATGATCGCCGTAGTAACGTCGCTGTCGTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGGCTAGTCCGTTATCAACTTGAAAAAGTGGCACCGAGTCGGTGGTGCTTTTTTTGTTTTTTATGTCTTCGAGTCATGTAATTAGTTAAGTGCAGGT
AGTGCAGGTTCTTTGAAAAGATAATGTATGATTATGCTTTCACTCATATTTATACAGAAACTTGATGTTTTCTTTCGAGTATATACAAGGTGATTACATGTACGTTTGAAGTACAACTCTAGATTTTGTAGTGCCCTCTTGGGCTAGCGGTAAAGGTGCGCATTTTTTCACACCCTACAATGTTCTGTTCAAAAGATTTTGGTCAAACGCTGTAGAAGTGAAAGTTGGTGCGCATGTTTCGGCGTTCGAAACTTCTCCGCAGTGAAAGATAAATGATCGCCGTAGTAACGTCGCTGTCGTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGGCTAGTCCGTTATCAACTTGAAAAAGTGGCACCGAGTCGGTGGTGCTTTTTTTGTTTTTTATGTCTTCGAGTCATGTAATTAGTTAATCGCGTG


In [22]:
U_pSNR52_Fw_1.annotations['batches']   = [{'location':'o3_E07'}]
TJOS_66_P2R.annotations['batches']   = [{'location':'o3_E08'}]
U_pSNR52_Fw_2.annotations['batches']   = [{'location':'o3_E09'}]
TJOS_65_P1R.annotations['batches']   = [{'location':'o3_F01'}]

In [23]:
sys.path.append('../constrain/design')
from cloning import USER_enzyme

gRNA1_pcr_USER = USER_enzyme(gRNA1_pcr_prod)
gRNA2_pcr_USER = USER_enzyme(gRNA2_pcr_prod)

In [24]:
vector.cut(AsiSI)

(Dseqrecord(-6972), Dseqrecord(-1830))

In [25]:
vector_asiSI, cCCDB,  = vector.cut(AsiSI)
vector_asiSI.seq

Dseq(-6972)
  CGCG..TGCGAT
TAGCGC..ACGC  

In [26]:
vector_asiSI.cut(BsmI)

(Dseqrecord(-8), Dseqrecord(-6960), Dseqrecord(-8))

In [27]:
BsmI_front,vector_asiSI_BsmI, BsmI_end = vector_asiSI.cut(BsmI)

In [28]:
BsmI_front.seq

Dseq(-8)
  CGCGTG
TAGCGC

In [29]:
front_overhang_add = BsmI_front.reverse_complement().seq.watson[::-1] + "acg"
front_overhang_add

'TAGCGCacg'

In [30]:
front_overhang_add_dseq = Dseqrecord(Dseq(crick = front_overhang_add,watson="c", ovhg = 0))[::-1]
front_overhang_add_dseq.seq

Dseq(-9)
        c
TAGCGCacg

In [31]:
BsmI_end.seq

Dseq(-8)
  TGCGAT
GCACGC

In [32]:
end_overhang_add = "gcg" + BsmI_end.seq.watson
end_overhang_add

'gcgTGCGAT'

In [33]:
end_overhang_add_dseq = Dseqrecord(Dseq(watson = end_overhang_add, crick="c", ovhg = 0))
end_overhang_add_dseq.seq

Dseq(-9)
gcgTGCGAT
c

In [34]:
vector_asiSI_BsmI.seq

Dseq(-6960)
  CATT..AATGCG
ACGTAA..TTAC  

In [35]:
vector_asiSI_BsmI_blunt = vector_asiSI_BsmI
vector_asiSI_BsmI_blunt.seq = vector_asiSI_BsmI.seq.T4('g')[1:-1]
vector_asiSI_BsmI_blunt.seq 

Dseq(-6954)
ATTC..GAAT
TAAG..CTTA

In [36]:
vector_overhangs = front_overhang_add_dseq + vector_asiSI_BsmI_blunt + end_overhang_add_dseq
vector_overhangs.seq

Dseq(-6972)
        cATT..AATgcgTGCGAT
TAGCGCacgTAA..TTAc        

In [37]:
vector_overhangs.seq.watson = vector_overhangs.seq.watson[:-1]
vector_overhangs.seq

Dseq(-6972)
        cATT..ATgccgTGCGA
TAGCGCaccgTA..TTAc       

In [38]:
gRNA1_pcr_USER.seq

Dseq(-425)
        TCTT..GTTAAGTGCAGGT
GCACGCTAAGAA..CAAT         

In [39]:
gRNA1_pcr_USER.seq.watson = gRNA1_pcr_USER.seq.watson[:-1]
gRNA1_pcr_USER.seq

Dseq(-425)
        TCTT..TTAAAGTGCAGG
GCACGCTAAAGA..CAAT        

In [40]:
gRNA2_pcr_USER.seq

Dseq(-425)
         TCTT..GTTAATCGCGTG
TCACGTCCAAGAA..CAAT        

In [41]:
rec_vec =  (vector_overhangs + gRNA1_pcr_USER + gRNA2_pcr_USER).looped()

TypeError: sticky ends not compatible!

OBS this process has changed gRNA1 feature locations one bp to the rigth. I have manually fixed these in Benchling as a workaround.

In [ ]:
rec_vec.name = "pCfB-56(pESC-LEU-gRNA_ATF1-CroCPR)"

In [ ]:
rec_vec.annotations['batches'] = {'location':'p1_G09'}

In [ ]:
sys.path.append('../constrain/design')
from cloning import extract_gRNAs

In [ ]:
gRNAs = extract_gRNAs(rec_vec, "gRNA")
gRNAs

Accounting for .looped() method movement of feature locations

In [ ]:
gRNAs[0].seq = 'g' + gRNAs[0].seq[:-1]
gRNAs[0]

In [ ]:
gRNAs[1].name

## PCR reactions

Phusion U

In [ ]:
gRNA1_pcr_prod.name = "ATF1"
gRNA1_pcr_prod.annotations['batches'] = [{'location':'l4_I07'}]
gRNA1_pcr_prod.annotations['template_location'] = "PL_05_D9"

gRNA2_pcr_prod.name = "CroCPR"
gRNA2_pcr_prod.annotations['batches'] = [{'location':'l4_I08'}]
gRNA2_pcr_prod.annotations['template_location'] = "p1_F04"

In [ ]:
sys.path.append('../constrain/lab')
from PCR import pcr_locations1

pcr_locations1([gRNA1_pcr_prod, gRNA2_pcr_prod])

In [ ]:
from PCR import pcr_volumes
pcr_volumes(vol_p_reac = 20, 
        no_of_reactions = 3,
        standard_reagents = ["Template", "Primer 1", "Primer 2", "H20", "Pol"],
        standard_volumes = [1, 2.5, 2.5, 19, 25])

In [ ]:
#gRNA1_pcr_prod.program() #.dbd_program

In [ ]:
len(gRNA1_pcr_prod)

In [ ]:
len(gRNA2_pcr_prod)

In [ ]:
#gel([PennStateLadder,[gRNA1_pcr_prod],[gRNA2_pcr_prod]]) #GeneRuler_1kb_; [Dseqrecord("A"*2000)]
gel([PennStateLadder,[Dseqrecord("A"*500)],[Dseqrecord("A"*500)], [vector]])

### Input picture from gel here: 

Image("..path/name_of_file.jpg", width = 100)

Plasmid contamination. Gel extraction.

In [ ]:
gRNA1_pcr_prod.annotations['batches'][0].update({'volume'       : 50,
                                                 'concentration': 50})
gRNA2_pcr_prod.annotations['batches'][0].update({'volume'       : 50,
                                                 'concentration': 50})

USER assembly

BB_01_C1 made by asisI and nb.bsmI treatment of p0056\(pESC-LEU-ccdB-USER)

l4_I06  = BB_01_C1


In [ ]:
LEU_plasmid = from_benchling("pESC-LEU-gRNA_CroCPR-2")
LEU_plasmid.name = "LEU_plasmid"
LEU_plasmid.annotations['batches'] = [{'location'     :'p1_F06',
                                       'volume'       : 30,
                                       'concentration': 470}]

In [ ]:
reaction_names = ["pESC-LEU-gRNA_ATF1-CroCPR", "n.ctr", "n.ctr", "n.ctr", "p. ctr"]

In [ ]:
reaction_participants = [[vector, gRNA1_pcr_prod,gRNA2_pcr_prod],
                         [vector],
                         [gRNA1_pcr_prod],
                         [gRNA2_pcr_prod], 
                         [LEU_plasmid]]

- 15 ng vector
- 30 ng gRNA
- 10 ng LEU plasmid

In [ ]:
sys.path.append('../constrain/lab')
#from Transformation import transf_locations1
from Transformation import ng_to_nmol, transformation_mix

In [ ]:
nmol_vector = ng_to_nmol(ng = 15, bp = len(vector))
nmol_gRNA = ng_to_nmol(ng = 30, bp = len(gRNA1_pcr_prod))
nmol_pctr = ng_to_nmol(ng = 10, bp = len(LEU_plasmid))

In [ ]:
# in nmol
wanted_concentrations = {'p0056\\(pESC-LEU-ccdB-USER)' : nmol_vector,
                         'ATF1'                        : nmol_gRNA,
                         'CroCPR'                      : nmol_gRNA,
                         'LEU_plasmid'                 : nmol_pctr}

In [ ]:
media = ['LB_AMP'] * 5

In [ ]:
transformation_mix(reaction_names, 
                  reaction_participants, 
                  wanted_amounts = wanted_concentrations,
                  water_dna_p_reac = 7,
                  media = media)

In [ ]:
pcr_volumes(vol_p_reac = 10, 
        no_of_reactions = 6,
        standard_reagents = ["DNA","Buffer, Cutsmart","H20","Enz, USER"],
        standard_volumes = [1,1,7,1])

|  | °C | min | 
| --- | --- | --- |
| Incubate | 37 | 25 | 
|  | 25 | 15 | 
|  | 0 | ... | 

Transformation:

Add 90 µl DH5a cells to 10 µl USER reaction & incubate

|  | °C | min | 
| --- | --- | --- |
| Incubate | 0 | 30 | 
|  | 42 | 1.5 (90s) | 
|  | 0 | 5 | 


### Plate on lb amp and insert picture of the transformation below: 
Image('../figures/transformations/name_of_file.jpg', width=600)

1. Pick 4 colonies from data (x example(2021-01-12)) plate no. 1
2. Plasmid prep
3. Sanger sequence from both ends

In [ ]:
ADH1_test_fw,CYC1_test_rv, = parse_primers(''' >ADH1_test_fw
                                      GAAATTCGCTTATTTAGAAGTGTC
                                      >CYC1_test_rv
                                      CTCCTTCCTTTTCGGTTAGAG''')

In [ ]:
ADH1_test_fw.annotations['batches']   = [{'location':'o1_C07'}]
CYC1_test_rv.annotations['batches']     = [{'location':'o1_C08'}]

| Colony no. | Primer_batch_location | Result|
| --- | --- |--- |
| 1 | o1_C07 | Correct |
| 2 | o1_C07 | Wrong |
| 3 | o1_C07 | Correct |
| 4 | o1_C07 | Bad trace |
| 1 | o1_C08 | Correct |
| 2 | o1_C08 | Wrong |
| 3 | o1_C08 | Correct |
| 4 | o1_C08 | Bad trace |

Conclusion: Colony no. 1 and 3 seems correct

Sample 1 becomes batch 001 in location p1_G09
Sample 3 becomes batch 002 in location p1_I01

In [ ]:
rec_vec.name

In [ ]:
rec_vec.annotations['batches'] = [{'location':['p1_G09']},{'location':['p1_I01']}]

In [ ]:
#rec_vec.write("../data/processed/p1_G09.gb")

In [ ]:
#rec_vec.to_benchling()...